<a href="https://colab.research.google.com/github/VellummyilumVinoth/Toxic_Comment_Classification/blob/main/Scrap_user_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#intall python reddit api wrapper
! pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 2.5 MB/s eta 0:00:00


In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import praw
from datetime import datetime, timedelta

In [3]:
import praw
import pandas as pd

reddit = praw.Reddit(
    client_id='4HprKxXtcLSkJ6uftWSzIg',
    client_secret='IN18nxBl_9vZrRQBTTLWrF7a5dewwA',
    user_agent='user_agent',
		check_for_async=False
)



In [4]:
import csv

# Create a list to store the data for each user
user_data = []

usernames = ['Orcasystems99', 'RevealDisinfo', 'Excellent-Issue3214', 'MantasChan', 'themimeofthemollies']  # Replace with the target user's usernames

current_date = datetime.now()
# Calculate the start and end dates for the desired week
start_date = (current_date - timedelta(days=current_date.weekday())).date()
end_date = start_date + timedelta(days=6)

# Create dictionaries to store the author names and titles separately
author_names = {}
post_titles = {}

for username in usernames:
    # Fetch the Redditor instance for the target user
    redditor = reddit.redditor(username)

    # Iterate over the user's submissions (posts)
    for submission in redditor.submissions.new(limit=None):
        post_date = datetime.fromtimestamp(submission.created_utc).date()
      
        if start_date <= post_date <= end_date:
            # Store the author name in the dictionary
            author_names.setdefault(username, []).append(username)
            
            # Store the post title in the dictionary
            post_titles.setdefault(username, []).append(submission.title)

# Combine the author names and post titles dictionaries into a single list
combined_data = []
for username in usernames:
    for i in range(len(author_names.get(username, []))):
        combined_data.append({
            'Username': author_names[username][i],
            'Post Title': post_titles[username][i]
        })

# Specify the CSV file path
csv_file = 'reddit_posts.csv'

# Write the user data to the CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    fieldnames = ['Username', 'Post Title']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()

    # Write the data rows
    for data in combined_data:
        writer.writerow(data)

print(f"Data saved to {csv_file}.")

Data saved to reddit_posts.csv.


In [7]:
from google.colab import files
files.download('reddit_posts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import csv
from textblob import TextBlob

# Load the CSV file
csv_file = 'reddit_posts.csv'

# Create a dictionary to store the toxic scores for each author
toxic_scores = {}

# Read the CSV file and calculate toxic scores
with open(csv_file, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        username = row['Username']
        post_title = row['Post Title']

        # Perform sentiment analysis on the post title
        blob = TextBlob(post_title)
        toxic_score = blob.sentiment.polarity

        # Update the toxic scores for the author
        toxic_scores.setdefault(username, []).append(toxic_score)

# Calculate the average toxic score for each author
average_scores = {}
for username, scores in toxic_scores.items():
    average_score = sum(scores) / len(scores)
    average_scores[username] = average_score

# Sort the authors based on the average toxic score
sorted_authors = sorted(average_scores.items(), key=lambda x: x[1], reverse=True)

# Display the sorted authors and their toxic scores
print("Ranking of Authors by Toxic Score:")
for rank, (author, score) in enumerate(sorted_authors, start=1):
    print(f"Rank {rank}: Author: {author}, Average Toxic Score: {score:.4F}")


Ranking of Authors by Toxic Score:
Rank 1: Author: RevealDisinfo, Average Toxic Score: 0.0695
Rank 2: Author: Orcasystems99, Average Toxic Score: 0.0226
Rank 3: Author: themimeofthemollies, Average Toxic Score: 0.0141
Rank 4: Author: MantasChan, Average Toxic Score: -0.0152
Rank 5: Author: Excellent-Issue3214, Average Toxic Score: -0.0206
